# Library

In [1]:
import numpy as np
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import torch
import argparse
import time
import pickle

#from src.self_awareness.networks import utils
#from src.self_awareness.learning.tf_cnn_auxiliary_gp import Model
from torch.distributions import Normal
import matplotlib.pyplot as plt
import random

import roslib
import rospy
import tf as tf_ros
from nav_msgs.msg import Odometry, Path
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from geometry_msgs.msg import PoseStamped, PoseArray, Pose
import math
import cv2
import copy

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


# Check GPU

In [2]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(1))

TITAN Xp


# Set torch default parameters

In [3]:
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4,sci_mode=False)
torch.backends.cudnn.benchmark = True

# Init ROS

In [4]:
rospy.init_node('global_localization_tf_broadcaster_cnn')

# Set Arguments

In [5]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=1, help='size of mini batch')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')

parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/dual_resnet_torch', help='model directory')

parser.add_argument('--test_dataset', type=str, default=[# '/notebooks/michigan_nn_data/2012_01_08',
                                                         # '/notebooks/michigan_nn_data/2012_01_15',
                                                         # '/notebooks/michigan_nn_data/2012_01_22',
                                                         # '/notebooks/michigan_nn_data/2012_02_02',
                                                         # '/notebooks/michigan_nn_data/2012_02_04',
                                                         # '/notebooks/michigan_nn_data/2012_02_05',
                                                         '/notebooks/michigan_nn_data/2012_02_12',
                                                         # '/notebooks/michigan_nn_data/2012_03_31',
                                                         '/notebooks/michigan_nn_data/2012_04_29',
                                                         '/notebooks/michigan_nn_data/2012_05_11',
                                                         '/notebooks/michigan_nn_data/2012_06_15',
                                                         '/notebooks/michigan_nn_data/2012_08_04',
                                                         # '/notebooks/michigan_nn_data/2012_09_28'])
                                                         '/notebooks/michigan_nn_data/2012_10_28',
                                                         '/notebooks/michigan_nn_data/2012_11_16',
                                                         '/notebooks/michigan_nn_data/2012_12_01'
                                                        ] )

parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
#parser.add_argument('--map_dataset', type=str, default='/home/kevin/data/michigan_gt/training')
sys.argv = ['']
args = parser.parse_args()

# Load Dataset

In [35]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
#from PIL import Image
import numpy as np
import random

#import gpflow.multioutput.kernels as mk
import gpytorch

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet
from torchlib.utils import LocalizationDataset
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.train_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False, mode='evaluate')
#[args.norm_mean, args.norm_std] = [torch.tensor(x) for x in dataset.get_norm()]
#args.norm_mean = torch.Tensor([-114.69805908,  405.21035767,   -8.72568321])
#args.norm_std = torch.Tensor([119.66057587, 176.14263916,   4.68300915])
[args.norm_mean, args.norm_std] = torch.load('/notebooks/global_localization/norm_mean_std.pt')

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=False, num_workers=0, \
                        drop_last=False, pin_memory=True)

100%|██████████| 5593/5593 [00:05<00:00, 967.67it/s] 


# Define Model

In [36]:
def denormalize_navie(normed_target, norm_mean, norm_std):
    target_trans_unscaled = normed_target * norm_std
    target_trans_uncentered = target_trans_unscaled + norm_mean
    
    return target_trans_uncentered

def denormalize(normed_target, norm_mean, norm_std):
    normed_target_trans, normed_target_rot = torch.split(normed_target, [3,4], dim=1)
    target_trans_unscaled = normed_target_trans * norm_std
    target_trans_uncentered = target_trans_unscaled + norm_mean
    target = torch.cat([target_trans_uncentered, normed_target_rot],dim=1)
    return target

def normalize(target, norm_mean, norm_std):
    target_trans = target[:,:3]
    target_trans = torch.div(torch.sub(target_trans,norm_mean),norm_std)
    target_normed = torch.cat([target_trans,target[:,3:]],dim=1)
    return target_normed 

class Model(nn.Module):
    def __init__(self,training=True):
        super().__init__()
        self.resnet = resnet.resnet50(pretrained=True) # dense_feat
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        #self.relative_context = vggnet(input_channel=4096,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        self.training = training
        
    def forward(self, input_data_t0, input_data_t1=None):
        if self.training:
            dense_feat0 = self.resnet(input_data_t0)
            dense_feat1 = self.resnet(input_data_t1)
            #dense_feat_relative = torch.cat([dense_feat0,dense_feat1],dim=1)

            global_context_feat0 = self.global_context(dense_feat0)
            global_context_feat1 = self.global_context(dense_feat1)
            #relative_context_feat = self.relative_context(dense_feat_relative)

            global_output0,_,_ = self.global_regressor(global_context_feat0)
            global_output1,_,_ = self.global_regressor(global_context_feat1)

            return global_output0,global_output1#,relative_context_feat 
        else:
            dense_feat = self.resnet(input_data_t0)
            global_context_feat = self.global_context(dense_feat)
            global_output,_,_ = self.global_regressor(global_context_feat)
            return global_output

In [ ]:
class CNN_Model:
    def __init__(self, training = True, device = "cpu"):
        # device
        self.device = torch.device(device)
        
        # data
        self.model = Model(training).to(device)
        self.norm_mean = args.norm_mean.to(device)
        self.norm_std = args.norm_std.to(device)
        
        # training tool
        self.optimizer = optim.Adam(self.model.parameters(), 
                                    lr=args.learning_rate, 
                                    weight_decay=args.weight_decay)
        self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                     lr_lambda=lambda epoch: args.decay_rate**epoch)
        
    def load_model(self, file_name = 'pretrained.pth', display_info = True):
        state_dict = torch.load(os.path.join(args.model_dir, file_name))
        if display_info:
            for name,param in state_dict.items():
                print(name, param.shape)
            print('Parameters layer:',len(state_dict.keys()))
        self.model.load_state_dict(state_dict,strict = False)
        
    def display_structure(self):
        for name, param in self.model.named_parameters():
            print(name, param.shape)
        print('Parameters layer:',len(self.model.state_dict().keys()))
    
    def display_require_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                print(name, param.shape)
    
    def power_resnet(self, status = False):
        if status = 'off':
            for param in self.model.resnet.parameters():
                param.requires_grad = False
        elif status = 'on':
            for param in self.model.resnet.parameters():
                param.requires_grad = True
        else:
            raise Exception("status must be 'on' or 'off'.")
            
    def power_context(self, status = False):
        if status = 'off':
            for param in self.model.global_context.parameters():
                param.requires_grad = False
        elif status = 'on':
            for param in self.model.global_context.parameters():
                param.requires_grad = True
        else:
            raise Exception("status must be 'on' or 'off'.")
    
    def power_regressor(self, status = False):
        if status = 'off':
            for param in self.model.global_regressor.parameters():
                param.requires_grad = False
        elif status = 'on':
            for param in self.model.global_regressor.parameters():
                param.requires_grad = True
        else:
            raise Exception("status must be 'on' or 'off'.")
            
    def save_model(self, file_name = 'model-{}-{}.pth'):
        checkpoint_path = os.path.join(args.model_dir, file_name)
        torch.save(self.model.state_dict(),checkpoint_path)
        print('saving model to' +  file_name)
            
    def loss(self,x0, x1, y0, y1):
        start = time.time()
        
        x0,x1,y0,y1 = x0.to(device),x1.to(device),y0.to(device),y1.to(device)
        y0_norm, y1_norm = [normalize(y,args.norm_mean, args.norm_std) for y in [y0,y1]]
        
        relative_target_normed = get_relative_pose(y0_norm, y1_norm)
        
        self.optimizer.zero_grad()
        
        global_output0,global_output1 = self.model(x0, x1)
        relative_consistence = get_relative_pose(global_output0,global_output1)
        global_loss = translational_rotational_loss(pred=global_output1, \
                                                    gt=y1_norm, \
                                                    lamda=args.lamda_weights)
        geometry_consistent_loss = translational_rotational_loss(pred=relative_consistence, \
                                                                 gt=relative_target_normed, \
                                                                 lamda=args.lamda_weights)
        total_loss = global_loss + geometry_consistent_loss        
        total_loss.backward()
        self.optimizer.step()
        
        end = time.time()
        batch_time = end - start
        return batch_time, float(total_loss)
        
        

In [37]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(device)

# set to cpu
#device = torch.device("cpu")
model = Model().to(device)
state_dict = torch.load(os.path.join(args.model_dir,'pretrained.pth'))
# pretrained
for name,param in state_dict.items():
    print(name, param.shape)
print('Parameters layer:',len(state_dict.keys()))

model.load_state_dict(state_dict,strict = False)

# Disable net
for param in model.parameters():
    param.requires_grad = False

resnet.conv1.weight torch.Size([64, 1, 7, 7])
resnet.bn1.weight torch.Size([64])
resnet.bn1.bias torch.Size([64])
resnet.bn1.running_mean torch.Size([64])
resnet.bn1.running_var torch.Size([64])
resnet.bn1.num_batches_tracked torch.Size([])
resnet.layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
resnet.layer1.0.bn1.weight torch.Size([64])
resnet.layer1.0.bn1.bias torch.Size([64])
resnet.layer1.0.bn1.running_mean torch.Size([64])
resnet.layer1.0.bn1.running_var torch.Size([64])
resnet.layer1.0.bn1.num_batches_tracked torch.Size([])
resnet.layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
resnet.layer1.0.bn2.weight torch.Size([64])
resnet.layer1.0.bn2.bias torch.Size([64])
resnet.layer1.0.bn2.running_mean torch.Size([64])
resnet.layer1.0.bn2.running_var torch.Size([64])
resnet.layer1.0.bn2.num_batches_tracked torch.Size([])
resnet.layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
resnet.layer1.0.bn3.weight torch.Size([256])
resnet.layer1.0.bn3.bias torch.Size([256])
resnet.layer1.0.bn3.ru

In [38]:
args.norm_mean = args.norm_mean.to(device)
args.norm_std = args.norm_std.to(device)

In [39]:
trans_errors = []
rot_errors = []
uncertainties = []
pose_map = []

total_trans_error = 0.
total_rot_error = 0.

count = 0.

is_save_map = False
is_read_map = False

trans_preds = []
trans_gts = []

rot_preds = []
rot_gts = []

pred_uncertainties = []

pred_time = []

In [40]:
br = tf_ros.TransformBroadcaster()

GT_POSE_TOPIC = '/gt_pose'
BIRDVIEW_TOPIC_PUB = '/bird_view'
MAP_TOPIC_PUB = '/pose_map'
PARTICLES_PUB = '/particles'
NN_LOCALIZASION_PUB = '/nn_pose'
gt_pose_pub = rospy.Publisher(GT_POSE_TOPIC, Odometry, queue_size=1)
bird_view_pub = rospy.Publisher(BIRDVIEW_TOPIC_PUB, Image, queue_size=1)
map_pub = rospy.Publisher(MAP_TOPIC_PUB, Path, queue_size=1)
particles_pub = rospy.Publisher(PARTICLES_PUB, PoseArray, queue_size=1)
nn_pose_pub = rospy.Publisher(NN_LOCALIZASION_PUB, Odometry, queue_size=1)

In [41]:
model.eval()

Model(
  (resnet): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2

In [53]:
def get_output(target_data_t1,global_output1,i=0):
    trans_target, rot_target = torch.split(target_data_t1, [3, 4], dim=1)
    global_output1_demormed = denormalize(global_output1, args.norm_mean, args.norm_std)
    trans_prediction, rot_prediction = torch.split(global_output1_demormed, [3, 4], dim=1)
    return trans_prediction, rot_prediction, trans_target, rot_target

for b, data in enumerate(dataloader, 0):
    start = time.time()
    x,y = data.values()
    x,y = x.to(device),y.to(device)
    #y = normalize(y,args.norm_mean, args.norm_std)
    
    # Get single data & transform data type
    with torch.no_grad():
        global_output1 = model(x)
    trans_pred, rot_pred, trans_gt, rot_gt = get_output(y,global_output1)
    trans_pred = np.asarray(trans_pred.cpu())
    rot_pred = np.asarray(rot_pred.cpu())
    trans_gt = np.asarray(trans_gt.cpu())
    rot_gt = np.asarray(rot_gt.cpu())
    x = np.asarray(x.cpu())
    
    end = time.time()
    
    br.sendTransform((trans_pred[0][0], trans_pred[0][1], trans_pred[0][2]),
                 (rot_pred[0][0], rot_pred[0][1], rot_pred[0][2], rot_pred[0][3]), rospy.Time.now(),
                 "estimation", "world")
    
    [px_gt, py_gt, pz_gt] = [trans_gt[0][0], trans_gt[0][1], trans_gt[0][2]]
    [qx_gt, qy_gt, qz_gt, qw_gt] = [rot_gt[0][0], rot_gt[0][1], rot_gt[0][2], rot_gt[0][3]]

    br.sendTransform((px_gt, py_gt, pz_gt),
                     (qx_gt, qy_gt, qz_gt, qw_gt),
                     rospy.Time.now(), "gt", "world")

    timestamp = rospy.Time.now()
    
    gt_msg = Odometry()
    gt_msg.header.frame_id = 'world'
    gt_msg.header.stamp = timestamp
    gt_msg.child_frame_id = 'base_link'
    gt_msg.pose.pose.position.x = px_gt
    gt_msg.pose.pose.position.y = py_gt
    gt_msg.pose.pose.position.z = pz_gt
    [gt_msg.pose.pose.orientation.x, gt_msg.pose.pose.orientation.y, gt_msg.pose.pose.orientation.z, gt_msg.pose.pose.orientation.w] = [qx_gt, qy_gt, qz_gt, qw_gt]

    bridge = CvBridge()

    bird_view_img_msg = bridge.cv2_to_imgmsg(np.asarray(x[0], dtype=np.float32), encoding="passthrough")
    stamp_now = rospy.Time.now()
    bird_view_img_msg.header.stamp = stamp_now

    bird_view_pub.publish(bird_view_img_msg)

    rospy.sleep(.1)

    count += 1
    
    trans_preds.append(trans_pred[0])
    rot_preds.append(rot_pred[0])
    trans_gts.append(trans_gt[0])
    rot_gts.append(rot_gt[0])

    trans_error = np.sum((trans_pred[0] - trans_gt[0])**2)**0.5
    rot_error_1 = np.arccos(np.dot(rot_pred[0], rot_gt[0]))/math.pi*180
    rot_error_2 = np.arccos(np.dot(rot_pred[0], -rot_gt[0])) / math.pi * 180
    rot_error = min(rot_error_1, rot_error_2)

    trans_errors.append(trans_error)
    rot_errors.append(rot_error)

    total_trans_error += trans_error
    total_rot_error += rot_error

    display = 50

    if b % display == 0 and b > 0:
        print(
            "{}/{}, translation error = {:.3f}, rotation error = {:.3f}, time/batch = {:.3f}"
            .format(
             b,
            len(dataloader),
            total_trans_error / count,
            total_rot_error / count,
            end - start))

print("pred time", np.mean(np.array(pred_time)))
print("time std", np.std(np.array(pred_time)))
    

50/5583, translation error = 1.608, rotation error = 3.870, time/batch = 0.009
100/5583, translation error = 2.292, rotation error = 4.308, time/batch = 0.009
150/5583, translation error = 2.583, rotation error = 4.645, time/batch = 0.009
200/5583, translation error = 2.954, rotation error = 4.422, time/batch = 0.009
250/5583, translation error = 3.276, rotation error = 4.316, time/batch = 0.009
300/5583, translation error = 3.334, rotation error = 4.443, time/batch = 0.009
350/5583, translation error = 4.056, rotation error = 4.938, time/batch = 0.009
400/5583, translation error = 4.368, rotation error = 4.818, time/batch = 0.009
450/5583, translation error = 4.733, rotation error = 4.736, time/batch = 0.009
500/5583, translation error = 4.995, rotation error = 4.637, time/batch = 0.009
550/5583, translation error = 4.910, rotation error = 4.481, time/batch = 0.009
600/5583, translation error = 4.887, rotation error = 4.301, time/batch = 0.009
650/5583, translation error = 4.796, rota

5150/5583, translation error = 3.280, rotation error = 3.638, time/batch = 0.009
5200/5583, translation error = 3.266, rotation error = 3.633, time/batch = 0.010
5250/5583, translation error = 3.260, rotation error = 3.627, time/batch = 0.009
5300/5583, translation error = 3.248, rotation error = 3.609, time/batch = 0.009
5350/5583, translation error = 3.234, rotation error = 3.594, time/batch = 0.010
5400/5583, translation error = 3.229, rotation error = 3.576, time/batch = 0.010
5450/5583, translation error = 3.214, rotation error = 3.559, time/batch = 0.009
5500/5583, translation error = 3.214, rotation error = 3.559, time/batch = 0.010
5550/5583, translation error = 3.220, rotation error = 3.558, time/batch = 0.009
pred time nan
time std nan


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [55]:
import scipy.io as sio

sio.savemat('results.mat', {'trans_pred': np.array(trans_preds), 'trans_gt': np.array(trans_gts), 'uncertainty': np.array(pred_uncertainties)})

if len(pose_map):
    np.savetxt(os.path.join(args.map_dataset, 'map.txt'), np.asarray(pose_map, dtype=np.float32))
    print("map is saved!")

plt.hist(trans_errors, bins='auto')
plt.title("Translation errors")
plt.xlabel("translational error in meters")
plt.ylabel("number of frames")
plt.savefig('terror.png', bbox_inches='tight')

plt.hist(rot_errors, bins='auto')
plt.title("Rotation errors")
plt.xlabel("rotational error in degree")
plt.ylabel("number of frames")
plt.savefig('rerror.png', bbox_inches='tight')

median_trans_errors = np.median(trans_errors)
median_rot_errors = np.median(rot_errors)
mean_trans_errors = np.mean(trans_errors)
mean_rot_errors = np.mean(rot_errors)

print("median translation error = {:.3f}".format(median_trans_errors))
print("median rotation error = {:.3f}".format(median_rot_errors))
print("mean translation error = {:.3f}".format(mean_trans_errors))
print("mean rotation error = {:.3f}".format(mean_rot_errors))   

median translation error = 2.472
median rotation error = 2.535
mean translation error = 3.225
mean rotation error = 3.547
